# Kostya's stuff

In [18]:
%pylab inline
%run ../utils.py
%run ../3DVariables/Functions.py

`%matplotlib` prevents importing * from pylab and numpy


Populating the interactive namespace from numpy and matplotlib
import failed


In [2]:
data = get_data()

NameError: name 'get_data' is not defined

In [3]:
from matplotlib.colors import LogNorm
import seaborn as s
s.set(rc={'image.cmap': "jet"})
config_plots(rcParams)

In [4]:
sel = lambda s: [ c for c in data.columns 
    if c.startswith(s) and c[-1] not in ["0","8"] ]

etrack = data[sel("EDepLayer")].mean(axis=1)
scatter(data['BetaTOF'], etrack, c='k', s=1, marker='.', lw=0)
plot(Beta_cent,ETrack,'b')
plot(Beta_cent,ETrack+sigmaEtrackinv*(ETrack**2),'r')
plot(Beta_cent,ETrack-sigmaEtrackinv*(ETrack**2),'r')
xlim(0.4,1.0); ylim(0,1)
xlabel("Beta TOF"); ylabel("mean EdepLayer in inner")

NameError: name 'data' is not defined

In [5]:
sel = lambda s: [ c for c in data.columns if c.startswith(s) ]

scatter(data['BetaTOF'], data[sel('EdepTOF')].mean(axis=1), c='k', s=1, marker='.', lw=0)
plot(Beta_cent,ETOF,'b')
plot(Beta_cent,ETOF+sigmaEtofinv*(ETOF**2),'r')
plot(Beta_cent,ETOF-sigmaEtofinv*(ETOF**2),'r')
xlim(0.4,1.0); ylim(0,10)
xlabel("Beta TOF"); ylabel("mean EdepTOF")

NameError: name 'data' is not defined

In [6]:
scatter(data['BetaTOF'], data['EdepTRD']/data['NTRDclusters'], c='k', s=1, marker='.', lw=0)
plot(Beta_cent,ETRD,'b')
plot(Beta_cent,ETRD+sigmaETRDinv*(ETRD**2),'r')
plot(Beta_cent,ETRD-sigmaETRDinv*(ETRD**2),'r')
xlim(0.4,1.0); ylim(0,10)
xlabel("Beta TOF"); ylabel("EdepTRD/NTRDclusters")

NameError: name 'data' is not defined

In [7]:
scatter(data['R'],data['BetaTOF'], c='k', s=1, marker='.', lw=0)
Rs = valorecent
Bs = Rs/np.sqrt(1.875**2+Rs**2)
plot(Rs,Bs,'b')
plot(Rs + sigmaRinv*(Rs**2),Bs,'r')
plot(Rs - sigmaRinv*(Rs**2),Bs,'r')
xlim(0,10); ylim(0.4,1)

NameError: name 'data' is not defined

In [8]:
scatter(data['R'],data['BetaTOF'], c='k', s=1, marker='.', lw=0)
Bs = betacent 
Rs = 1.875*Bs/np.sqrt(1-Bs**2)
plot(Rs,Bs,'b')
plot(Rs,Bs + sigmabetainv*(Bs**2),'r')
plot(Rs,Bs - sigmabetainv*(Bs**2),'r')
xlim(0,10); ylim(0.4,1)

NameError: name 'data' is not defined

In [9]:
Beta_cent

NameError: name 'Beta_cent' is not defined

# Benoît's stuff

## Comparison between observed target matrix and the MCMC predicted matrix

The following plot is the target matrix obtained with real data.
The cut used are:
[
    "physicsTrigger",
    "chargeOne",
    "oneTrack",
    "goldenTOF",
    "goldenTRACKER",
    "oneParticle",
    "goldenTRD",
    "betaNotCrazy"]

x axis is rigidity measured and y axis is beta measured


<img src="files/protonObserved.png">

The following plot is the MCMC predicted matrix taking into account only protons (ie. the deuton flux is set to 0). Is is obtained by taking the proton parameters where the MCMC has converged and build the expected beta Vs rig distribution with them. **It is based on AMS.protonsB800**

<img src="files/protonPredicted.png">

Let's look at the difference between the predicted distribution and the observed one.
The following plog is : (predicted - observed)/observed

<img src="files/protonPredictedMinusObserved.png">

The enveloppes added to the plot represents the observed distribution
We are not interested by the red blob which is a region that will be masked. We are only interested by the region where the enveloppes are.

What we can see is **there is an excess of proton in the peak of the distribution and there is a lack of proton in the tail. This can be explained by the fact that the sigma of beta TOF in the Monte-Carlo is too small.**

(incidently we can also see the blue region where the deutons are missing. It is normal since no deutons where injected in the MCMC)

## investigating sigma

The following plot shows the mean value of beta TOF as a function of the measured rigidity for data and MC. It is obtained by vertically slicing the aforementioned hObserved and hPredicted distributions and fitting every slice by a gaussian. The mean of every gaussian is the y-value of every point.
<img src="files/betaTOFMeanVsRig.png">
The agreement between both graphs is good.

However this is not the case for the sigma distribution.
<img src="files/betaTOFSigmaVsRig.png">

# onclusion

**At least in protonsB800 the beta TOF sigma is wrong by ~12%**

In [10]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import glob

%pylab inline

import os


Populating the interactive namespace from numpy and matplotlib


In [11]:
def loadBinary(dirname):
    print 'loading : ' + dirname
    data=dict()
    for file in os.listdir(dirname):
        if file.startswith("time_L") or file.startswith("R_") or file.startswith('betaH_')  or file.startswith('beta_'):
            data[file.split('_chunk')[0]] = np.fromfile(dirname+'/'+file, np.dtype('float32'))

    return DataFrame(data)

def makeDataframe(files, name):
    df = pd.concat(map(loadBinary, files))
    df = df[(df.time_L0 > -999) & (df.time_L1 > -999) & (df.time_L2 > -999) & (df.time_L3 > -999) & (df.R < 15) & (df.R > 1) ]
    df['binRgdt'] = df['R'].astype('int')
    df.index.name = name
    return df

dfMC = makeDataframe( glob.glob('/data/dst/mc/*.00000001.output'), "MC" )
dfData = makeDataframe( glob.glob('/data/dst/data/*.00000001.output'), "Data" )


    


loading : /data/dst/data/1306467229.00000001.output


loading : /data/dst/data/1306482583.00000001.output


loading : /data/dst/data/1306473829.00000001.output


loading : /data/dst/data/1306475549.00000001.output


loading : /data/dst/data/1306483092.00000001.output

...


loading : /data/dst/mc/872759191.00000001.output


loading : /data/dst/mc/872759188.00000001.output
loading : /data/dst/mc/872759189.00000001.output


loading : /data/dst/mc/872759190.00000001.output


loading : /data/dst/mc/872733375.00000001.output

loading : /data/dst/mc/872733373.00000001.output
loading : /data/dst/mc/872759193.00000001.output

In [20]:
ax=dict()
fig=dict()
rigBins=range(1,8)
ranges = dict()
# ranges['time_L3-time_L2'] = (-1,1.5)
ranges['time_L3-time_L1'] = (2,10)
#     ranges['time_L3-time_L0'] = (2,10)
    
for rigBin in rigBins:
    fig[rigBin] = dict()
    for var in ranges:
        fig[rigBin][var] = figure()
        plt.title(var + ' energy : [{},{}]'.format(rigBin,rigBin+1))


def plotTofTimeDifferences(df):

    layers=['time_L2','time_L1','time_L0']
    for l in layers: df['time_L3-'+l]=df['time_L3']-df[l]
    for var in ranges:
        for rigBin in rigBins:
            plt.figure(fig[rigBin][var].number)
            n, bins, patches = plt.hist(df[df['binRgdt']==rigBin][var].values,label=df.index.name,normed=True, bins=200, range=ranges[var],histtype='step')
            xdata, ydata = ((bins[:-1] + bins[1:])*0.5,n)
            out = fit(gaussian,xdata,ydata,[0.5,5,1])
            x,y = zip(*filter(lambda x: -out[0][2] < x[0]-out[0][1] < out[0][2],zip(xdata,ydata)))
            out = gaussianFit(x,y,out[0])
            plt.plot(x,gaussian(out[0],x),linewidth=2.0)

#         yscale('log')
    
plotTofTimeDifferences(dfMC)
plotTofTimeDifferences(dfData)

for f in fig.itervalues():
    for f_ in f.itervalues():
        plt.figure(f_.number)
        legend()

In [ ]:
import bigQueryPlotting as bq


import math

def binning(df,var,nBins,firstBin,lastBin):
    binWidth=(lastBin-firstBin)/nBins
    return firstBin + binWidth * (df[var]-firstBin).floordiv(binWidth)

def plotDataFrame2D(df,nBinsX,firstBinX,lastBinX,nBinsY,firstBinY,lastBinY,varX,varY):
    binX='bin_'+varX
    binY='bin_'+varY
    dfTmp=pd.DataFrame(df[[varX,varY]])
    dfTmp[binX]=binning(dfTmp,varX,nBinsX,firstBinX,lastBinX)
    dfTmp[binY]=binning(dfTmp,varY,nBinsY,firstBinY,lastBinY)
    dfTmp=dfTmp[(dfTmp[binX] >= firstBinX) & (dfTmp[binX] < lastBinX) & (dfTmp[binY] >= firstBinY) & (dfTmp[binY] < lastBinY) ]
    plot_matrix( dfTmp.groupby([binY,binX]).count()[varX].unstack().fillna(0) )

for df in [dfData,dfMC]:
    print df.index.name
    df['invBetaH']=1/df.betaH
    figure()
    plotDataFrame2D(df,100,0.9,1.6,100,2,7.,'invBetaH','time_L3-time_L1')
    


Beta distribution of MC and data is similar when integrated over the rigidity

In [ ]:
for df in [dfData,dfMC]:
    df['betaH'].hist(normed=True,range=(0.8,1.1),bins=100,histtype='step')

However if we look per rigidity slice, this is not the case:

In [58]:
def makeHistAndFit(data,init):
    n, bins, patches = plt.hist(data,normed=True,range=(0.6,1.1),bins=100,histtype='step',linewidth=2.0)
    xdata, ydata = ((bins[:-1] + bins[1:])*0.5,n)
    out = fit(gaussian,xdata,ydata,init)
    x,y = zip(*filter(lambda x: -out[0][2] < x[0]-out[0][1] < out[0][2],zip(xdata,ydata)))
    out = fit(gaussian,x,y,out[0])
    plt.plot(x,gaussian(out[0],x),linewidth=2.0)
    return (out[0][1],out[0][2])

def fitAndGetSigmaForAllRigBin(data):
    betaPar = []
    for rig in rigBins:
        figure(fig['rig : '+str(rig)].number)
        betaPar.append( makeHistAndFit(df[df['binRgdt']==rig]['betaH'].values, [1.0, 0.5, 0.5]) )
    return betaPar

fig=dict()
for rig in rigBins:
    fig['rig : '+str(rig)]=figure()
    plt.title('rig : '+str(rig))


mean = dict()
std = dict()
for df in [dfData,dfMC]:
    mean[df.index.name],std[df.index.name]=zip(* fitAndGetSigmaForAllRigBin(df) )

figure()
plot(rigBins, mean['Data'])
print rigBins
print mean['Data']

[1, 2, 3, 4, 5, 6, 7]
(0.83478493254997777, 0.9371203518623713, 0.9617580286006262, 0.97157160243865914, 0.97714728098202452, 0.98254439080746681, 0.9859983783308951)


In [ ]:
figure()
plt.title('Standard deviation betaH: Data/MC')
plt.plot(rigBins,[x/y for x,y in zip(betaPar['Data']['std'],betaPar['MC']['std'])])

